In [5]:
import collections
from collections import Counter
from fcmeans import FCM
import json
import math
import numpy as np
import os
import os
import pyedflib
import pywt
import random
import re
import scipy.stats
from scipy.stats import entropy, tstd, tmean
from scipy import signal
import sklearn.cluster
from sklearn.cluster import Birch
from sklearn.cluster import KMeans
from sklearn import metrics
from tabulate import tabulate
import time
import warnings

In [8]:
def under_sample(data):
  normal_index = data[:,1] == 0
  abnormal_index = data[:,1] == 1

  normal = data[normal_index, :]
  abnormal = data[abnormal_index, :]

  rand_index = np.random.choice(normal.shape[0], abnormal.shape[0], replace=False)
  normal_sample = normal[rand_index, :]

  dataset = np.concatenate((abnormal, normal_sample))
  np.random.shuffle(dataset)

  return np.asarray([x for x in dataset[:,0]]), np.asarray(dataset)[:,1]

In [9]:
################################################################################
# Helper functions to compute the features
################################################################################

def calculate_ave_power(coeffs):
    ave_power = np.sum(np.square(coeffs)) / len(coeffs)
    return ave_power

def calculate_mean(coeffs):
    mean = scipy.stats.tmean(coeffs)
    return mean

def calculate_entropy(coeffs):
    counter = Counter(coeffs).most_common()
    pk = [x[1] / len(coeffs) for x in counter]
    entropy = scipy.stats.entropy(pk)
    return entropy

def calculate_std_dev(coeffs):
    std_dev = scipy.stats.tstd(coeffs)
    return std_dev

def get_features(coeffs):
    ave_power = calculate_ave_power(coeffs)
    mean = calculate_mean(coeffs)
    std_dev = calculate_std_dev(coeffs)
    entropy = calculate_entropy(np.asarray(coeffs))

    return [ave_power, mean, std_dev, entropy]

################################################################################
# Perform multi-level 1D Discrete Wavelet Transform (DWT)
# Approximation and coefficients of the previous level is the input to the
# current level
################################################################################

def get_dwt_features(signal):
    coeffs = pywt.wavedec(signal, 'db2', level=2)
    return get_features(np.concatenate(coeffs))

# Validation Methods

In [10]:
def is_abnormal(index, seg_size, durations):
  for duration in durations:
    seizure_start = duration[0]
    seizure_end = duration[1]
    seg_start = index
    seg_end = index + seg_size

    if seizure_start <= seg_start <= seizure_end or seg_start <= seizure_start <= seg_end:
      return True

  return False

def segment(signal):
    seg_size = 1280 # 5 Seconds
    overlap = 0.1 # 90% overlap
    index = 0
    segments = []

    need_last_seg = True

    while index <= len(signal) - 1280:
      if index + seg_size == len(signal) - 1:
        need_last_seg = False

      segment = signal[index:index + seg_size]
      features = get_dwt_features(segment)
      segments.append(features)
      
      index += math.ceil(overlap * seg_size)

    if need_last_seg:
      segment = signal[-seg_size:]
      segments.append(get_dwt_features(segment))

    return np.asarray(segments)

def load_single_file(edf_file):
  X_data = []
  f = pyedflib.EdfReader(edf_file)
  n = f.signals_in_file
  for i in range(n):
    signal = f.readSignal(i, digital=True)
    segments = segment(signal)
    X_data.append(segments)
  f.close()
  return np.asarray(X_data)

def calculate_seziure_times(labels, tolerance=1):
  start = None
  count = 0
  seizures = []
  for (index, label) in enumerate(labels):
    if label:
      if not start:
        start = index * 128
    else:
      if start:
        count += 1
        if count == tolerance:
          seizures.append((start, index * 128))
          start = None
          count = 0
  return seizures

In [11]:
def test_cmeans(edf_file):
  warnings.filterwarnings('ignore')
  #dataset = json.load(open(r'C:\Users\kscot\Documents\TestResults\CHB01\Dataset_CHB01.json'))
  dataset = json.load(open(r'/Users/fatimasiddiqui/Documents/School/Sem1/ECE496/Dataset_CHB01.json'))

  train_set = dataset[:2] + dataset[3:]
  X_train = [i[1] for i in train_set]
  X_train = [j for i in X_train for j in i]

  Y_train = [i[2] for i in train_set]
  Y_train = [j for i in Y_train for j in i]

  X_train, Y_train = under_sample(np.array(list(zip(X_train, Y_train))))
  
  ideal_k = 4
  cmeans = FCM(n_clusters=ideal_k)
  cmeans.fit(X_train)

  cluster_assignments = cmeans.predict(X_train)
  contingency_matrix = metrics.cluster.contingency_matrix(Y_train, cluster_assignments)

  label_mapping = np.argmax(contingency_matrix, axis=0)

  ################################################################################
  # Start Timer
  start_time = time.time()

  X_test = load_single_file(edf_file)
  all_seizure_times = []
  ret = ""
  
  all_seizure_start_times = []
  #test_f = open(r"C:\Users\kscot\Documents\chb-mit-scalp-eeg-database-1.0.0\out.csv", "a")
  test_f = open(r"/Users/fatimasiddiqui/Documents/School/Sem1/ECE496/results/out.csv", "a")

  for index, signal in enumerate(X_test):
    ret += "Channel " + str (index + 1) + ": "
    predictions = cmeans.predict(signal)
    labels = [label_mapping[i] for i in predictions]

    seizure_times = calculate_seziure_times(labels)
    seizure_start_times = "Ch" + str(index + 1)

    for seizure_time in seizure_times:
      ret += "Start: " + str(seizure_time[0]) + ", End: " + str(seizure_time[1]) + "; "
      seizure_start_times += "," + str(seizure_time[0])

    seizure_start_times += "\n"
    test_f.write(seizure_start_times)
    ret += "\n\n"

  test_f.close()

  # End Timer
  end_time = time.time()
  elapsed_time = end_time - start_time
  ################################################################################

  ret += "\n\nElapsed Time: " + str(elapsed_time)
  print(all_seizure_start_times)

  #test_f = open(r"C:\Users\kscot\Documents\chb-mit-scalp-eeg-database-1.0.0\out.txt", "w")
  test_f = open(r"/Users/fatimasiddiqui/Documents/School/Sem1/ECE496/results/out.txt", "w")
  test_f.write(ret)
  test_f.close()

  return ret

In [ ]:
# Python program to create
# a file explorer in Tkinter
  
# import all components
# from the tkinter library
from tkinter import *
  
# import filedialog module
from tkinter import filedialog
  
# Function for opening the
# file explorer window

def browseFiles():
    filename = filedialog.askopenfilename(initialdir = "/",
                                          title = "Select a File",
                                          filetypes = (("EDF Files",
                                                        "*.edf"),
                                                       ("all files",
                                                        "*.*")))
    
    if filename:
      label_file_explorer.configure(text="Loading...")
      out = test_cmeans(filename)
      # Change label contents
      #label_file_explorer.configure(text="File Opened: "+filename)
      label_file_explorer.configure(text=out)
    
      
                                                                                                  
# Create the root window
window = Tk()
  
# Set window title
window.title('Seizure Detection')
  
# Set window size
#window.geometry("520x520")
  
#Set window background color
window.config(background = "white")
  
# Create a File Explorer label
label_file_explorer = Label(window,
                            text = "Select an EDF File",
                            width = 60, height = 27,
                            fg = "blue")
  
      
button_explore = Button(window,
                        text = "Browse Files",
                        command = browseFiles)
  
# Grid method is chosen for placing
# the widgets at respective positions
# in a table like structure by
# specifying rows and columns
label_file_explorer.grid(column = 1, row = 1)
  
button_explore.grid(column = 1, row = 2)
  
# Let the window wait for any events
window.mainloop()

[]
